In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import os, sys
%matplotlib inline

os.chdir('..\..\..\..')

from input_configuration import base_year

from IPython.display import display, HTML

if (sys.version_info < (3, 0)):
    HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
working_dir = r'outputs/agg/dash'
# pd.options.display.float_format = '{:0,.0f}'.format
format_percent = "{:,.2%}".format

In [3]:
def load_data(fname):
    """ open and join model and survey data """

    df_model = pd.read_csv(os.path.join(working_dir,fname+'.csv'))
    df_model['source'] = 'model'
    df_survey = pd.read_csv(os.path.join(working_dir,'survey',fname+'.csv'))
    df_survey['source'] = 'survey'
    df = df_model.append(df_survey)

    return df

In [4]:
parcel_geog = pd.read_sql_table('parcel_'+base_year+'_geography', 'sqlite:///inputs/db/soundcast_inputs.db',
        columns=['CountyName','ParcelID'])

In [5]:
def dist_by_mode(purpose=None):
    df = load_data('tour_distance')
    # Trim any unreasonable survey distances off by taking max distance as max modeled bin value
    max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
    df = df.loc[df['tautodist_bin'] <= max_model_dist]
    df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
    df_dist = pd.DataFrame()
    for source in ['model','survey']:
        if purpose is not None:
            _df = df[(df['source'] == source) & (df['pdpurp'] == purpose)]
        else:
            _df = df[(df['source'] == source)]
        df1 = _df.groupby('tmodetp').sum()[['wt_autodist']].reset_index()
        df2 = _df.groupby('tmodetp').sum()[['toexpfac']].reset_index ()
        _df = df1.merge(df2, on='tmodetp')
        _df['commute_distance'] = _df['wt_autodist']/_df['toexpfac']
        _df['source'] = source
        df_dist = df_dist.append(_df)
    df = df_dist.pivot_table(index='tmodetp', columns='source', aggfunc='first', values='commute_distance')
    df.rename(columns={'tmodetp': 'Tour Mode'}, inplace=True)
    for col in ['School Bus',' Park']:
        if (col in df.index) & (purpose != 'School'):
            df.drop(col, axis=0, inplace=True)
        if (col != 'School Bus') & (col in df.index):
            df.drop(col, axis=0, inplace=True)
    
    return df

### Total

In [6]:
df = load_data('tour_distance')
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df = df[(df['pdpurp'] == 'School')]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df = df.groupby('source').sum()
pd.DataFrame(df['wt_autodist']/df['toexpfac'], columns=['School Distance (miles)'])

,School Distance (miles)
source,
model,3.667215
survey,4.283469


### By Purpose

In [7]:
df = load_data('tour_distance')
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df_dist = pd.DataFrame()
for source in ['model','survey']:
    _df = df[(df['source'] == source)]
    df1 = _df.groupby('pdpurp').sum()[['wt_autodist']].reset_index()
    df2 = _df.groupby('pdpurp').sum()[['toexpfac']].reset_index ()
    _df = df1.merge(df2, on='pdpurp')
    _df['commute_distance'] = _df['wt_autodist']/_df['toexpfac']
    _df['source'] = source
    df_dist = df_dist.append(_df)
df = df_dist.pivot_table(index='pdpurp', columns='source', aggfunc='first', values='commute_distance')
df.rename(columns={'pdpurp': 'Tour Purpose'}, inplace=True)
df.drop('Change Mode Inserted Purpose', axis=0, inplace=True)
df

source,model,survey
pdpurp,,
Escort,6.998221,3.884350
Meal,2.817477,4.223924
Personal Business,6.765277,5.486210
School,3.667215,4.283469
Shop,4.733719,4.527161
Social,4.470941,4.889143
Work,11.439409,11.508843


# Tour Distance
### All Tours

In [8]:
dist_by_mode()

source,model,survey
tmodetp,,
Bike,5.182160,3.032501
HOV2,6.968551,6.698868
HOV3+,6.643900,6.216336
Park,15.785402,24.393105
SOV,8.496981,8.497786
TNC,2.333954,3.992971
Transit,9.409102,9.402977
Walk,0.931334,0.889081


### Average Work Tour Distance By Mode

In [9]:
dist_by_mode('Work')

source,model,survey
tmodetp,,
Bike,5.565858,4.307769
HOV2,11.469063,11.350996
HOV3+,11.841534,15.006290
Park,15.785402,23.257848
SOV,12.022697,11.779248
TNC,6.006925,4.858663
Transit,12.911899,10.774342
Walk,0.762817,1.290412


### Average School Tour Distance by Mode

In [10]:
dist_by_mode('School')

source,model,survey
tmodetp,,
Bike,2.569636,1.295566
HOV2,3.522687,5.358035
HOV3+,3.642029,3.623373
Park,NaN,28.455959
SOV,7.355436,6.082790
School Bus,3.193878,3.464268
TNC,2.332774,2.772132
Transit,5.969371,7.658118
Walk,1.112619,0.948390


### Average Personal Business Tour Distance by Mode

In [11]:
dist_by_mode('Personal Business')

source,model,survey
tmodetp,,
Bike,3.265017,3.049242
HOV2,7.823077,6.479523
HOV3+,7.781722,6.014153
Park,NaN,19.282414
SOV,6.614503,5.332477
TNC,1.995497,4.930529
Transit,6.696441,6.090867
Walk,1.182023,0.899835


### Average Shopping Tour Distance by Mode

In [12]:
dist_by_mode('Shop')

source,model,survey
tmodetp,,
Bike,2.408022,0.561337
HOV2,5.474640,4.915027
HOV3+,5.629333,6.499530
Park,NaN,0.500000
SOV,4.688517,3.756609
TNC,1.472745,2.642004
Transit,4.368060,2.258857
Walk,1.084966,0.630863


### Average Meal Tour Distance by Mode

In [13]:
dist_by_mode('Meal')

source,model,survey
tmodetp,,
Bike,1.080935,2.683948
HOV2,4.645685,4.585905
HOV3+,4.978334,6.294448
Park,NaN,30.114636
SOV,2.804167,4.501964
TNC,1.208114,2.337364
Transit,3.070070,8.467957
Walk,0.686159,0.704359


### Average Social Tour Distance by Mode

In [14]:
dist_by_mode('Social')

source,model,survey
tmodetp,,
Bike,2.479569,2.334264
HOV2,6.010500,7.382173
HOV3+,6.003816,4.983953
Park,NaN,54.147252
SOV,5.339749,5.652026
TNC,1.412438,3.019705
Transit,5.090444,11.227402
Walk,0.953371,0.882807


### Average Escort Tour Distance by Mode

In [15]:
dist_by_mode('Escort')

source,model,survey
tmodetp,,
Bike,12.383043,0.647842
HOV2,6.493710,4.363578
HOV3+,7.078159,3.749272
SOV,21.570943,3.974768
TNC,1.602665,0.500000
Transit,2.221212,2.256245
Walk,0.958653,0.552118


Survey Sample Size Reference by Mode and Purpose

In [16]:
# Survey Sample size
df_tour_survey = pd.read_csv(r'inputs/base_year/survey/_tour.tsv', delim_whitespace=True)

In [17]:
df_tour_survey.pivot_table(index='tmodetp',columns='pdpurp', values='toexpfac', aggfunc='count')

pdpurp,1,2,3,4,5,6,7,10
tmodetp,,,,,,,,
1,889.0,189.0,158.0,375.0,631.0,811.0,1613.0,NaN
2,234.0,38.0,8.0,19.0,19.0,19.0,81.0,NaN
3,3215.0,114.0,43.0,890.0,820.0,352.0,937.0,1.0
4,945.0,284.0,637.0,560.0,587.0,373.0,743.0,NaN
5,455.0,442.0,477.0,336.0,287.0,246.0,659.0,1.0
6,2293.0,257.0,26.0,253.0,146.0,115.0,314.0,1.0
7,101.0,2.0,NaN,7.0,1.0,4.0,10.0,NaN
8,9.0,253.0,1.0,20.0,NaN,NaN,7.0,NaN
9,119.0,12.0,1.0,29.0,6.0,42.0,49.0,NaN


# Work Tour Distance by Worker Type

In [18]:
df = load_data('tour_distance')
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df_dist = pd.DataFrame()
for source in ['model','survey']:
    _df = df[(df['source'] == source) & 
             (df['pdpurp'] == 'Work') & 
             (df['pptyp'].isin(['Full-Time Worker','Part-Time Worker','University Student']))]
    df1 = _df.groupby('pptyp').sum()[['wt_autodist']].reset_index()
    df2 = _df.groupby('pptyp').sum()[['toexpfac']].reset_index()
    _df = df1.merge(df2, on='pptyp')
    _df['commute_distance'] = _df['wt_autodist']/_df['toexpfac']
    _df['source'] = source
    df_dist = df_dist.append(_df)
df = df_dist.pivot_table(index='pptyp', columns='source', aggfunc='first', values='commute_distance').reset_index()
df.rename(columns={'pptyp': 'Person Type'}, inplace=True)
df

source,Person Type,model,survey
0,Full-Time Worker,11.974886,12.146355
1,Part-Time Worker,8.664870,7.607614
2,University Student,7.387945,9.322525
